In [1]:
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip -q install "langchain[docarray]"
!pip show langchain langchain-core
!pip -q install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metada

1. https://huggingface.co/blog/gemma
2. Repo model google/gemma-7b-it is gated. You must be authenticated to access it.
3. Create your 'new token' on your huggingface

pip install torch accelerate transformers langchain chroma sentence-transformers langchain_community pypdf

In [14]:
#@title Configuration de l'authentification
import os
import google.generativeai as genai
from google.colab import userdata
import json
os.environ["GOOGLE_API_KEY"] = userdata.get('gemini')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [21]:
import os
import json
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader

import os
import json
from langchain.document_loaders import PyPDFLoader

safety_settings = [
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel(model_name="gemini-pro", safety_settings=safety_settings)

prompt_template = """
You are a medical expert assistant. Answer the question as detailed as possible from the provided context,
make sure to provide all the details, if the answer is not in provided context just write 'Not specified',
and give me this information in a JSON format ,don't provide the wrong answer.

Context:
{context}

Question:
{question}

Answer:
"""

folder_path = "/content/pdfs"

question = '''\
Please provide the following information in a JSON format with the specified keys:

- Patient_Information:
  - Name
  - Age
  - Clinical_Presentation:
    - OMS
    - Conjonctives
    - Ictère
    - Symptoms
    - Cicatrice_de_mastectomie
    - Sein_gauche
    - Sein_droit
    - ADP_infra_centimétrique_gauche (true or false)
  - Medical_History:
    - CCI_du_sein:
      - Type
      - Treatment:
        - CMT_néo_adjuvante
        - Patey
    - Anapath_Results:
      - Date
      - Results
  - Treatment_Information:
    - Pneumo_Treatment_Prescribed
    - Rhumato_Consultation_Missed (true or false)
    - Hormonothérapie
    - ITG  (authorized or not)
  - Imaging_Results:
    - Echo_Morpho:
      - Date
      - Results (as List)
    - Echo_Mamo:
      - Date
      - Results (as List)
    - TDM_TAP:
      - Date
      - Results (as List)
    - TAP+MT (true or false)
    - Echographie_Pelvienne (true or false)
    - Ostéo_Densitométrie (true or false)
    - DIU (true or false)
'''

patient_data_list = []

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path)
        clinical_context = loader.load()
        print(clinical_context)

        formatted_prompt = prompt_template.format(context=clinical_context, question=question)
        response = model.generate_content(formatted_prompt)

        cleaned_json_str = response.text.strip('```json\n').strip()  # Access the 'content' attribute before stripping
        json_dict = json.loads(cleaned_json_str)

        json_dict["file_name"] = filename
        patient_data_list.append(json_dict)

        print(f"Medical information from {filename} added to the list")
        print()

with open('patient_data.json', 'w') as json_file:
    json.dump(patient_data_list, json_file, indent=4, ensure_ascii=False)

print("Toutes les informations médicales ont été enregistrées dans : patient_data.json")


[Document(page_content="1 \n 15/12/2023 14:33:03  \nONCOLOGIE --> CONSULTATION ( LABORATOIRE )  \nMOUNIR BELCADI ABBASSI  \nPatiente se présente pour controle:   \n \nCliniquement:  \nOMS1 , conjonctives colorées , pas d'ictère  \nPF: Toux chronique, patiente vue par un pneumo à titre externe, traitement prescrit non acheté par la patiente  \nCicatrice de mastectomie propre et calme  \nSein gauche : RAS.  \nSein droit sans particularité.  \nLa patiente rapporte toujours de douleurs osseuses diffus es ===> Patiente a raté son RDV pour avis rhumato.  \nEcho mamo faite le 11/12/2023 :  \n\uf0b7 Lésion kystique au niveau de la JQExt du sein droit, classé ACR3  \n\uf0b7 Absence d'anomalie suspecte en regard du site de Patey  \nCAT:  \nOn continu le controle  \nPatiente indiscipliné ==> réadresser pour avis rhumato  \n  \n19/09/2023 15:34:40   \nONCOLOGIE --> CONSULTATION ( HOPITAL DE JOUR )   \nAGARAOUI MARYEM   \nPatiente se présente pour controle:   \n \n \nCliniquement:  \n \nOMS1 , conj